# Using Keras

This is some example code of how we can get the code in Keras, adapted from https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/.

In [4]:
import numpy as np
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [5]:
#an example dataset from the Keras tutorial
dataset = loadtxt('pima_indians.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

In [6]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### Compilation

This is a bit of a funny one, as we define an optimizer in this part. In actuality, though, this is only because we want to use Keras to do loss evaluation, which requires doing _model.compile_. We need to pass an optimizer argument to this, but we actually never use it because we don't call _model.train_

In [7]:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

### Getting stuff into the format we want

Basically, we can simply set the weights of the neural net using the function _model.set_weights_. We can then evaluate the objective function for a subset of data using _model.evaluate(InputData, TrueOutput)_. Pseudocode for getting the objective function for a particular position would then look something like this:

In actuality, the _get_weights_ function actually passes us a list of numpy arrays, so we have to do some manipulations, but this is pretty chill, and we don't really need to worry about it. Basically, what has happened is:

1) Extract the information about the shape of the data we get from the _model.get_weights_ function (stored in ls_form)

In [8]:
ls_form = []
my_weights = model.get_weights()
index = 0
for comp in my_weights:
    ls_form.append([index, index + np.size(comp), comp.shape])
    index += np.size(comp)

2) Define a function for converting the format we get from keras into one long numpy array

In [9]:
def Convert(pre_w):
    position = []
    for w in pre_w:
        position.append(w.flatten())
    return np.concatenate(position)

3) Define a function for converting it back again. This uses the information we stored in the list ls_form.

In [10]:
def ConvertBack(position):
    global ls_form    
    reinput = []
    for i in ls_form:
        reinput.append(position[i[0]:i[1]].reshape(i[2]))
    return reinput

### Validation

Just to check that this works, let's take the weights, convert them into one long numpy array and back again.

In [11]:
x_0 = model.get_weights()

In [12]:
b = Convert(x_0)
print(b.shape)

(221,)


In [10]:
x_1 = ConvertBack(b)

In [11]:
model.set_weights(x_1)

In [12]:
for i in range(len(ls_form)):
    print(np.allclose(x_0[i], x_1[i]))

True
True
True
True
True
True


## Model evaluation

We evaluate the loss associated with a model as such:

In [13]:
model.evaluate(X,y,verbose=0)[0]

28.67880415916443

Let's see how long this takes:

In [14]:
%timeit model.evaluate(X,y,verbose=0)[0]

36.5 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


This gives us the time it takes, on average, to calculate the loss associated with those weights for the entire dataset

#### Footnotes

* To clarify, neuron biases are included in the definition of "weights"
* When calculating the loss, we are using the loss funtion defined in _model.compile_ (in this case, binary cross-entropy)